In [3]:
from huggingface_hub import hf_hub_download, HfApi, ModelFilter, snapshot_download, login
from dotenv import load_dotenv
import os

REPO_ID = 'scvi-tools/MODEL-FOR-UNIT-TESTING-1'
API_TOKEN = os.getenv("API_TOKEN")

# Repository Setup

To use this repository, you need to set up a few things locally. Please follow the steps below:

### 1. Create `.env` File

1. Create a file named `.env` in the root directory of this repository.

2. Open the `.env` file in a text editor.

3. Add your Hugging Face API key to the `.env` file using the following format:

   ```plaintext
   API_TOKEN="your_api_key_here"

### 2. Creata a /local_models directory

1. Create a /local_models directory in the root of the scvi-hub directory. 
This is where the local models from scvi-hub will be stored.


In [4]:
login(token=API_TOKEN)

### List the models under a repository

In [5]:
api = HfApi()
models = api.list_models(
    filter=ModelFilter(
        author="scvi-tools",
    )
)

# Get all model IDs that do not have the words demo or testing in the name
filtered_model_ids = [model.modelId for model in models if "demo" not in model.modelId.lower() and "testing" not in model.modelId.lower()]

# Get all the model ids
print(filtered_model_ids)
# Get a certain model id
print(filtered_model_ids[1])


['scvi-tools/human-lung-cell-atlas', 'scvi-tools/tabula-sapiens-bladder-scvi', 'scvi-tools/tabula-sapiens-bladder-scanvi', 'scvi-tools/tabula-sapiens-bladder-condscvi', 'scvi-tools/tabula-sapiens-bladder-stereoscope', 'scvi-tools/tabula-sapiens-blood-scvi', 'scvi-tools/tabula-sapiens-blood-scanvi', 'scvi-tools/tabula-sapiens-blood-condscvi', 'scvi-tools/tabula-sapiens-blood-stereoscope', 'scvi-tools/tabula-sapiens-bone_marrow-scvi', 'scvi-tools/tabula-sapiens-bone_marrow-scanvi', 'scvi-tools/tabula-sapiens-bone_marrow-condscvi', 'scvi-tools/tabula-sapiens-bone_marrow-stereoscope', 'scvi-tools/tabula-sapiens-eye-scvi', 'scvi-tools/tabula-sapiens-eye-scanvi', 'scvi-tools/tabula-sapiens-eye-condscvi', 'scvi-tools/tabula-sapiens-eye-stereoscope', 'scvi-tools/tabula-sapiens-fat-scvi', 'scvi-tools/tabula-sapiens-fat-scanvi', 'scvi-tools/tabula-sapiens-fat-condscvi', 'scvi-tools/tabula-sapiens-fat-stereoscope', 'scvi-tools/tabula-sapiens-heart-scvi', 'scvi-tools/tabula-sapiens-heart-scanvi', 

### Download the models and cache them

The models will be downloaded in the local_models directory. The Pytorch files can be found under the /snapshots folder.

In [25]:
# Download files to local folder and use the cached version
# This example downloads the repository that has model_id[4]
# The model will be downloaded under "/snapshot"
model_path = "/app/scvi_hub_models/"

folder_path = snapshot_download(repo_id=filtered_model_ids[1], allow_patterns=["*.h5ad","*.pt","*.json"],cache_dir=model_path)


import json
import gdown

json_path = folder_path + "/_scvi_required_metadata.json"
f = open(json_path)

data = json.load(f)

training_data_url = data.pop("training_data_url")
model_parent_module = data.pop("model_parent_module")
model_cls_name = data.pop("model_cls_name")
output = folder_path + '/reference_atlas.h5ad'

gdown.download(training_data_url, output)

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Downloading...
From: https://zenodo.org/api/files/fd2c61e6-f4cd-4984-ade0-24d26d9adef6/TS_Bladder_filtered.h5ad
To: /app/scvi_hub_models/models--scvi-tools--tabula-sapiens-bladder-scvi/snapshots/89565a92a1504e9b22d73071c6e014e24b7d32c0/reference_atlas.h5ad
100%|██████████| 1.13G/1.13G [21:53<00:00, 861kB/s] 


'/app/scvi_hub_models/models--scvi-tools--tabula-sapiens-bladder-scvi/snapshots/89565a92a1504e9b22d73071c6e014e24b7d32c0/reference_atlas.h5ad'

### Start the ML-pipeline

Inject the model and reference atlas in the existing ML-pipeline

In [24]:
import scvi
import scarches as sca
import scanpy

reference = scanpy.read_h5ad(folder_path + "/reference_atlas.h5ad")
query = scanpy.pp.subsample(reference, 0.1, copy=True)

model = eval(model_parent_module + "." + model_cls_name).load_query_data(
            query,
            folder_path,
            freeze_dropout=True,
        )

model.train(
    max_epochs=10,
    plan_kwargs=dict(weight_decay=0.0),
    check_val_every_n_epoch=10,
    use_gpu=False
)


combined_adata = query.concatenate(reference, batch_key='bkey')
eval(model_parent_module + "." + model_cls_name).setup_anndata(combined_adata, batch_key="batch")

combined_adata.obsm["latent_rep"] = model.get_latent_representation(combined_adata)

/usr/local/lib/python3.9/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


AttributeError: module 'scvi.model' has no attribute 'RNAStereoscope'